# Fix existing AMBlob-s and AMBlobReference-s 
2022-12-04

The blobs wrapped by the AMBlobs have no file name/blob_id. Want to set this to the &lt;checksum&gt;.&lt;filetype&gt;.


In [ ]:
import sys
import os
import io
import pprint
# import lxml
import lxml.etree as ET
import xml.dom.minidom
import importlib
import glob
import json
import string
import datetime
import pandas
import getpass
import matplotlib.pyplot as plt
import requests
from PIL import Image, ImageDraw, ImageFont

In [ ]:
import pyxb
pyxb.RequireValidWhenGenerating(False);

# preliminaries

In [ ]:
import myconfig

In [ ]:
importlib.reload(myconfig)
from myconfig import *

In [ ]:
try: CONFIG.USER
except: CONFIG.USER = input('username: ')
try: CONFIG.PASS
except: CONFIG.PASS = getpass.getpass('enter password ')
AUTH=(CONFIG.USER,CONFIG.PASS)

In [ ]:
sys.path.insert(0, CONFIG.pyUTILS_path)
import ambench.cdcs_utils
from ambench.mapping import new_mapper
from ambench import amdoc

In [ ]:
importlib.reload(ambench.cdcs_utils)
from ambench.cdcs_utils import *

# create AMBench2022 instance

In [ ]:
AMBENCH_URL = CONFIG.AMBENCH_URL
AMBENCH_URL='https://test-ambench2022.nist.gov/'
ambench2022=AMBench2022(CONFIG.TEMPLATE,AMBENCH_URL,auth=AUTH)
# create validator from in-memory XSDs
CDCS_VALIDATOR = ambench2022.create_schema_validator(); 

# Retrieve the blobs and reload those whose filename == 'blob'

### load all AMBlob-s and update their handle to a blob with filename == [checksum].[format]

todo
* retrieve all blobs
* retrieve all AMBlobs
* for all blobs with filename 'blob' load the blob with filename based no the checksum and the format

In [ ]:
adf=ambench2022.query_all_amblobs()
updated_amblobs = {}
for t in adf.itertuples():
    el = xpath(t.xml_content,'/AMBlob')[0]
    handle = el.xpath("//*/handle/text()")[0]
    checksum = el.xpath("//*/checksum/text()")[0]
    r=requests.get(handle,auth=AUTH,verify=False)
    blobbytes = r.content
    b_checksum=checksum4bytes(blobbytes)  # hashlib.md5(r.content).hexdigest()
    if b_checksum != checksum:
        print("ERROR")
    updated_amblobs[checksum] = handle

In [ ]:
updated_amblobs

In [ ]:
def checksum4handle(handle):
    r=requests.get(handle_old,auth=AUTH,verify=False)
    blobbytes = r.content
    return checksum4bytes(blobbytes) 

In [ ]:
amdocs = ambench2022.query_all_docs()

In [ ]:
updated = []
for t in amdocs.itertuples():
    el=xpath(t.xml_content,'//AMDoc')[0]
    pid=el.xpath("pid/text()")[0]
    amblobrefs=el.xpath("//*/checksum/..")
    is_updated=False
    if len(amblobrefs) > 0:
        for amblobref in amblobrefs:
            d = {c.tag:c.text for c in amblobref}
            handle_old = d['handle']
            checksum_old = checksum4handle(handle_old)
            handle_new = updated_amblobs[d['checksum']]
            checksum_new = checksum4handle(handle_new)
            if checksum_old != checksum_new:
                print("Issue with",t.title)
            else:
                amblobref.find('handle').text = handle_new
                is_updated = True
    if is_updated:
        updated.append((t,ET.tostring(el, encoding="unicode")))

In [ ]:
for t,xml_content in updated[1:]:
    print(t.title)
    ambench2022.update_data(filename=t.title,title=t.title,content=xml_content)

## Assign all blobs to global workspace

In [ ]:
ambench2022.get_workspaces()

In [ ]:
blobs = ambench2022.get_blobs()
blobs_to_move = blobs[(blobs['filename'] != 'blob') & (~blobs["filename"].str.contains('.QR.'))& (blobs["filename"].str.contains('\.'))]

In [ ]:
blobs_to_move

In [ ]:
ambench2022.assign_blobs(workspace='Global Public Workspace', ids=blobs_to_move['id'].values)